In [5]:
import numpy as np
import math
import matplotlib.pyplot as plt  
from sklearn.metrics import mean_squared_error
import pandas as pd

from keras.layers import Dropout
from keras.layers import Dense,Input
from keras.models import Sequential,Model
from keras.callbacks import EarlyStopping
import numpy

import sys
import xlwt
##get data##
import MySQLdb
from sklearn.preprocessing import MinMaxScaler


def transfromData(trainRaw, testRaw,windosSize):  ##Train ratial, train, test
    sc = MinMaxScaler(feature_range = (0, 1))
    #npRaw = np.array(rawData)
    #scaledData = sc.fit_transform(npRaw.reshape(-1,1))  
    #trainData = np.array(scaledData[:trainCount]).reshape(-1,1)##train 
    #testData = np.array(scaledData[-testCount:]).reshape(-1,1)
    
    npTrain = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    npTest = sc.fit_transform(np.array(trainRaw).reshape(-1,1))
    

    X_train, y_train = splitXy(npTrain,windosSize)
    X_test, y_test = splitXy(npTest,windosSize)
    return sc, X_train, y_train, X_test, y_test

def splitXy(data,windosSize):
    windows = []
    for i in range(windosSize, data.shape[0]):
        windows.append(data[i-windosSize:i, 0])
    np.random.shuffle(windows)
    X = []
    y = []
    for i in range(len(windows)):
        X.append(windows[i][:windosSize-1])
        y.append(windows[i][-1:])
    X, y = np.array(X), np.array(y)
    #X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    return X,y
def buildModel(shape):
    dnn_1_input = Input(shape=(shape,),name='dnn_1_input')
    dnn_1 = Dense(128,activation="tanh", name='dnn_1')(dnn_1_input)
    dnn_2 = Dense(64,activation="tanh", name='dnn_2')(dnn_1)
    dnn_3 = Dense(32,activation="tanh", name='dnn_3')(dnn_2)
    dnn_4 = Dense(1, activation='tanh', name='dnn_4')(dnn_3)

    model = Model(dnn_1_input,dnn_4)
    model.summary()
    model.compile(loss='mse',
                  optimizer='adam')
    return model
def getStationList(cursor):
    stationList = []
    queryString = "SELECT distinct station from cleandata"
    cursor.execute(queryString)
    results = cursor.fetchall()
    for row in results :
        val = str(row).strip("',()")
        stationList.append(val)
    stationList.remove('富貴角')##only 106少數資料
    return stationList
def Visualize():
    predicted = sc.inverse_transform(regressor.predict(X_test))
    originY = sc.inverse_transform (y_test)
    print("MSE : ["+str(mean_squared_error(predicted, originY))+"]")
    # Visualising the results
    plt.plot(originY[:100], color = 'red', label = 'Real')  
    plt.plot(predicted[:100], color = 'blue', label = 'Predicted ') 
    plt.legend()
    plt.show()
def writeExcelHead(sheet1,epochs,station):
    sheet1.write(0,1,station)
    raw = 1
    for e in range(epochs):
        sheet1.write(raw,0,e+1)
        raw+=1


def fetchData(cursor,station,windosSize):
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date NOT like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    trainRawData = []
    for row in results:
        for i in range(3,27):
            trainRawData.append(float(row[i]))
            
    queryString = "SELECT * from cleandata where station='"+station+"' and item = 'PM2.5' and date like '2017%'"
    cursor.execute(queryString)
    results = cursor.fetchall()

    testRawData = []
    for row in results:
        for i in range(3,27):
            testRawData.append(float(row[i]))

    sc, X_train, y_train, X_test, y_test = transfromData(trainRawData,testRawData,windosSize)
    return sc, X_train, y_train, X_test, y_test
def train(regressor,sc, X_train, y_train, X_test, y_test,epochs):
 
    for i in range(epochs):
        regressor.fit(X_train, y_train,validation_split=0.2, epochs = 1, batch_size = 32,verbose=2)
        predicted = sc.inverse_transform(regressor.predict(X_test))
        originY = sc.inverse_transform (y_test)
        mse = mean_squared_error(predicted, originY)
        print("Epoch : " +str(i+1)+", MSE : ["+str(mse)+"]")
        print('-------------------------------------------')
    return mse

In [ ]:
import datetime
db=MySQLdb.connect(host="localhost",user="root", passwd="swater0", db="airdb", charset="utf8")
cursor = db.cursor()

epochs = 200
stationList = getStationList(cursor)
col=1

for station in stationList:
    print("training : " +station)
    MSEs = []
    for windowSize in range(2,12):
        sc, X_train, y_train, X_test, y_test = fetchData(cursor,station,windowSize)
        regressor = buildModel(windowSize-1)
        mse = train(regressor,sc,X_train, y_train, X_test, y_test,epochs)
        MSEs.append(mse)
    print('check point at ' + str(datetime.datetime.now()))
    
    book = xlwt.Workbook(encoding="utf-8")
    sheet1 = book.add_sheet("Sheet1")
    writeExcelHead(sheet1,epochs,station)
    row = 1
    for m in MSEs:
        sheet1.write(row,1,m)
        row+=1
    book.save("excelFiles/ANN/ANNresult"+station+".xls")
        
    

db.close()


training : 二林
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dnn_1_input (InputLayer)     (None, 1)                 0         
_________________________________________________________________
dnn_1 (Dense)                (None, 128)               256       
_________________________________________________________________
dnn_2 (Dense)                (None, 64)                8256      
_________________________________________________________________
dnn_3 (Dense)                (None, 32)                2080      
_________________________________________________________________
dnn_4 (Dense)                (None, 1)                 33        
Total params: 10,625
Trainable params: 10,625
Non-trainable params: 0
_________________________________________________________________
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 3s - loss: 0.0011 - val_loss: 9.7585e-04
Epoch : 1, MSE : [49.

 - 1s - loss: 8.3148e-04 - val_loss: 9.3108e-04
Epoch : 39, MSE : [47.65959137733725]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.3439e-04 - val_loss: 0.0010
Epoch : 40, MSE : [54.10250508354459]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.3561e-04 - val_loss: 9.3735e-04
Epoch : 41, MSE : [47.80885565103449]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.3752e-04 - val_loss: 9.4297e-04
Epoch : 42, MSE : [48.47401223777459]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2977e-04 - val_loss: 9.3590e-04
Epoch : 43, MSE : [47.59182550879292]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.3703e-04 - val_loss: 9.2790e-04
Epoch : 44,

 - 1s - loss: 8.2536e-04 - val_loss: 9.2577e-04
Epoch : 83, MSE : [47.19123343766575]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.3066e-04 - val_loss: 9.3765e-04
Epoch : 84, MSE : [48.0925184071639]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2312e-04 - val_loss: 9.3985e-04
Epoch : 85, MSE : [48.113470312078064]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.3044e-04 - val_loss: 9.2906e-04
Epoch : 86, MSE : [47.578784607138175]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2787e-04 - val_loss: 9.6894e-04
Epoch : 87, MSE : [49.966620153885906]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2754e-04 - val_loss: 9.3671e-04
Epoch

Epoch : 126, MSE : [47.06981272282812]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.1928e-04 - val_loss: 9.2256e-04
Epoch : 127, MSE : [47.130588176238405]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2249e-04 - val_loss: 9.3734e-04
Epoch : 128, MSE : [47.947290892267226]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2201e-04 - val_loss: 9.8099e-04
Epoch : 129, MSE : [49.85467755761358]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2275e-04 - val_loss: 9.3329e-04
Epoch : 130, MSE : [47.88533525315699]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2347e-04 - val_loss: 9.8181e-04
Epoch : 131, MSE : [50.835734150480704]
--------

 - 1s - loss: 8.1870e-04 - val_loss: 9.2041e-04
Epoch : 170, MSE : [46.88625771832864]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2259e-04 - val_loss: 9.2889e-04
Epoch : 171, MSE : [47.25606989064682]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.1990e-04 - val_loss: 9.2977e-04
Epoch : 172, MSE : [47.39937394993992]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2032e-04 - val_loss: 9.2422e-04
Epoch : 173, MSE : [47.17767679651312]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2161e-04 - val_loss: 9.2142e-04
Epoch : 174, MSE : [47.05733608271472]
-------------------------------------------
Train on 17144 samples, validate on 4286 samples
Epoch 1/1
 - 1s - loss: 8.2082e-04 - val_loss: 9.2427e-04
Ep

...

 - 3s - loss: 4.7090e-04 - val_loss: 5.1740e-04
Epoch : 58, MSE : [30.4223389765771]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.7325e-04 - val_loss: 4.7278e-04
Epoch : 59, MSE : [27.752556176775585]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.7257e-04 - val_loss: 4.5186e-04
Epoch : 60, MSE : [26.73283033322832]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.7923e-04 - val_loss: 4.4098e-04
Epoch : 61, MSE : [26.006628818479847]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.7495e-04 - val_loss: 4.8205e-04
Epoch : 62, MSE : [28.44981036061292]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.7236e-04 - val_loss: 4.3775e-04
Epoch 

Epoch : 101, MSE : [25.732398848914297]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.5978e-04 - val_loss: 4.2648e-04
Epoch : 102, MSE : [25.575150259710274]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.6121e-04 - val_loss: 5.6157e-04
Epoch : 103, MSE : [33.46963255247106]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.6195e-04 - val_loss: 4.2623e-04
Epoch : 104, MSE : [25.38977732337326]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.5946e-04 - val_loss: 4.6235e-04
Epoch : 105, MSE : [27.692019006160436]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.6212e-04 - val_loss: 4.4572e-04
Epoch : 106, MSE : [26.47940159334369]
--------

Epoch : 144, MSE : [25.525820171534896]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.5676e-04 - val_loss: 4.3008e-04
Epoch : 145, MSE : [25.643134278882993]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.5562e-04 - val_loss: 4.8654e-04
Epoch : 146, MSE : [28.83328688510641]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.5891e-04 - val_loss: 4.4753e-04
Epoch : 147, MSE : [26.585638480766804]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.5662e-04 - val_loss: 4.2426e-04
Epoch : 148, MSE : [25.40824163526097]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.5476e-04 - val_loss: 4.7490e-04
Epoch : 149, MSE : [28.342130578147515]
-------

 - 3s - loss: 4.5464e-04 - val_loss: 4.2067e-04
Epoch : 188, MSE : [25.208027007154982]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.4738e-04 - val_loss: 4.4044e-04
Epoch : 189, MSE : [26.260791495418573]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.5130e-04 - val_loss: 4.1514e-04
Epoch : 190, MSE : [24.82492253103536]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.5268e-04 - val_loss: 4.1956e-04
Epoch : 191, MSE : [25.201757212986493]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.4943e-04 - val_loss: 4.4876e-04
Epoch : 192, MSE : [26.612316663384323]
-------------------------------------------
Train on 21976 samples, validate on 5495 samples
Epoch 1/1
 - 3s - loss: 4.5135e-04 - val_loss: 4.3144e-0

 - 3s - loss: 4.7074e-04 - val_loss: 5.1882e-04
Epoch : 26, MSE : [28.130034707196515]
-------------------------------------------
Train on 21976 samples, validate on 5494 samples
Epoch 1/1
 - 3s - loss: 4.7169e-04 - val_loss: 4.9053e-04
Epoch : 27, MSE : [26.360535970513347]
-------------------------------------------
Train on 21976 samples, validate on 5494 samples
Epoch 1/1
 - 3s - loss: 4.7413e-04 - val_loss: 4.9038e-04
Epoch : 28, MSE : [26.401069148617054]
-------------------------------------------
Train on 21976 samples, validate on 5494 samples
Epoch 1/1
 - 3s - loss: 4.6794e-04 - val_loss: 5.3353e-04
Epoch : 29, MSE : [28.559367887345267]
-------------------------------------------
Train on 21976 samples, validate on 5494 samples
Epoch 1/1
 - 3s - loss: 4.7147e-04 - val_loss: 5.1402e-04
Epoch : 30, MSE : [27.967940551697193]
-------------------------------------------
Train on 21976 samples, validate on 5494 samples
Epoch 1/1
 - 3s - loss: 4.7422e-04 - val_loss: 5.1463e-04
Ep

In [5]:
X_train.shape

(21422, 9, 1)